# Projet Méthode Numérique - Option Lookback

In [118]:
import numpy as np
#import pylab as pl
from random import gauss #, uniform, randint
from math import exp,sqrt,log
from scipy.stats import norm
import math
import time
import wand

In [110]:
class Option_pricer:
    def price_call(self,price, strike,volatility , maturity,interest_rate ,dt):
        tps1 = time.clock()
        x, var = 0, 0

        IC = [0,0]
        for i in range(N):
            Ft = price
            for i in range(int(maturity/dt)):
                dW = gauss(0.0,sqrt(dt))
                Ft = Ft * exp((interest_rate-0.5*volatility**2)*dt + volatility*dW)
            Prix = exp(-interest_rate*maturity)*max(Ft-strike,0)  
            x += Prix
            var += Prix**2
        var /= N
        x /= N
        var -= x**2
        IC[1] = x + 2*sqrt(var/N)
        IC[0] = x - 2*sqrt(var/N)
        tps2 = time.clock()
        timer = tps2 - tps1
        return x, var, IC, timer
    
    def price_call_RV(self,price, strike,volatility , maturity,interest_rate ,dt):
        tps1 = time.clock()
        x, var = 0,0
        IC = [0,0]
        P = N/2
        for i in range(int(P)):
            lnFt = math.log(price)
            lnFt2 = math.log(price)
            for i in range(int(maturity/dt)):
                dW = gauss(0,1)
                lnFt = lnFt + (interest_rate - 0.5*volatility**2)*dt + volatility*sqrt(dt)*dW
                lnFt2 = lnFt2 + (interest_rate - 0.5*volatility**2)*dt - volatility*sqrt(dt)*dW
            Ft = exp(lnFt)
            Ft2 = exp(lnFt2)
            Prix = 0.5*(max(Ft2-strike,0) + max(Ft-strike,0))
            Prix = exp(-interest_rate*maturity)*Prix
            x += Prix
            var += Prix**2
        x /= P
        var = var/P - x**2
        IC[1] = x + 2*sqrt(var/P)
        IC[0] = x - 2*sqrt(var/P)
        tps2 = time.clock()
        timer = tps2 - tps1
        return x, var, IC,timer 

    
    def lookback_call_bs(self,price,strike,volatility,maturity,interest_rate):
        tps1 = time.clock()
        x, var = 0, 0
        min_st = 100
        IC = [0,0]
        for i in range(N):
            Ft = price
            for i in range(int(maturity/dt)):
                dW = gauss(0.0,sqrt(dt))
                Ft = Ft * exp((interest_rate-0.5*volatility**2)*dt + volatility*dW)
            if Ft < min_st :
                min_st = Ft
            Prix = np.exp(-interest_rate*maturity)*max(Ft-min_st,0)  
            x += Prix
            var += Prix**2
        var /= N
        x /= N
        var -= (x**2)
        IC[1] = x + 2*sqrt(var/N)
        IC[0] = x - 2*sqrt(var/N)
        tps2 = time.clock()
        timer = tps2 - tps1
        return x, IC, var, timer
    
    def lookback_call_RV(self,price,strike,volatility,maturity,interest_rate):
        tps1 = time.clock()
        x, var = 0, 0
        min_st,min_st2 = 100,100
        IC = [0,0]
        P = N/2
        for i in range(int(P)):
            Ft,Ft2 = price,price
            for i in range(int(maturity/dt)):
                dW = gauss(0.0,sqrt(dt))
                Ft = Ft * exp((interest_rate-0.5*volatility**2)*dt + volatility*dW)
                Ft2 = Ft2 * exp((interest_rate-0.5*volatility**2)*dt - volatility*dW)
            if Ft < min_st :
                min_st = Ft
            if Ft2 <min_st2 :
                min_st2 = Ft2
            Prix = np.exp(-interest_rate*maturity)*0.5*(max(Ft-min_st,0)+max(Ft2-min_st2,0))
            x += Prix
            var += Prix**2
        var /= P
        x /= P
        var -= x**2
        IC[1] = x + 2*sqrt(var/N)
        IC[0] = x - 2*sqrt(var/N)
        tps2 = time.clock()
        timer = tps2 - tps1
        return x, IC, var, timer       
    
    def __init__(
    self, price, strike, volatility, maturity,
    interest_rate, dt):
        self.price = price
        self.strike = strike
        self.volatility = volatility
        self.maturity = maturity
        self.interest_rate = interest_rate
        self.dt = dt
        self.price_call_MC,self.sd,self.IC,self.timerMC = self.price_call(price, strike, volatility,maturity,
                              interest_rate, dt)
        self.price_call_RV,self.sdRV,self.ICRV,self.timerMCRV= self.price_call_RV(price, strike, volatility,maturity,
                              interest_rate, dt)
        self.price_lookback_call,self.IClb,self.sdlb,self.timerLB = self.lookback_call_bs(price,strike,volatility,maturity,
                                                                             interest_rate)
        self.price_lookback_call_RV,self.IClb_RV,self.sdlb_RV,self.timerLBRV = self.lookback_call_RV(price,strike,volatility,
                                                                                      maturity,interest_rate)

# Jeu MC1 :

# exercice 1 : (ref photo)

In [123]:
#ref Fichier pdf 

# exercice 2

In [78]:
#CONTEXTE
N = 1000 # le nombre d'essai
interest_rate = 0.01 # les tx d'intérêts 
volatility = 0.2 # la volatilité, constante
dt = 1/12 # pas mensuel

In [124]:
price = 100
maturity = 2
strike = 100

ec = Option_pricer(price,strike,volatility,maturity,interest_rate,dt)
print("prix :" ,ec.price_call_MC,";variance :",ec.sd,";IC : ",ec.IC,"timer : ",ec.timerMC,"\n")
print("prix avec RV :",ec.price_call_RV,"; variance :",ec.sdRV,";IC :" ,ec.ICRV,"timer : ",ec.timerMCRV,"\n")

C:\Users\schut\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\schut\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\schut\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\schut\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\schut\Anaconda3\lib\si

prix : 13.453482315106097 ;variance : 457.4743682453602 ;IC :  [12.100745284866502, 14.806219345345692] timer :  0.0847036000341177 

prix avec RV : 12.541430240311785 ; variance : 137.4054378170472 ;IC : [11.492982100184493, 13.589878380439076] timer :  0.06772079999791458 



C:\Users\schut\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Le produit avec Réduction de variance bénéficie avant tout d'une reduction de temps car on génère deux fois moins de nombre aléatoire pour une même taille d'échantillon, et en plus de ça on est 2 à 3 fois plus efficace en terme de variance.

Analyse Qantitative du produit : 


Il existe deux types de lookback option, ici pour rappel on parle d'un floating strike puisque le strike ainsi que le prix sont volatiles. Toutefois c'est la valeur minimale du strike qu'on va prendre le maximal du stock dans un put et minimal dans un call.  
C'est pour ça que l'on dit que lorsqu'on achète ce produit exotique on devient long en delta et donc en volatilité et en risk free rate car plus l'actif sous jacent fluctue plus il devient potentiellement bénéficiaire.
Il faudrait modéliser la volatilité du sous jacent (stochastique) car c'est ça qui donne la valeur du produit.

In [125]:
ec = Option_pricer(price,strike,volatility,maturity,interest_rate,dt)

print("prix lookback:" ,ec.price_lookback_call,"; variance :",ec.sdlb,ec.IClb,"timer : ",ec.timerLB,"\n ")
print("prix lookback avec RV :",ec.price_lookback_call_RV,"; variance :",ec.sdlb_RV,ec.IClb_RV,"timer : ",ec.timerLBRV)

C:\Users\schut\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\schut\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\schut\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\schut\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\schut\Anaconda3\lib\si

prix lookback: 58.705212639744474 ; variance : 955.4005421721699 [56.750320783735894, 60.66010449575305] timer :  0.08341439999639988 
 
prix lookback avec RV : 49.941787622153576 ; variance : 83.97969704869502 [49.36220260837667, 50.52137263593048] timer :  0.05509749997872859


# MC2

In [6]:
#CONTEXTE : 
N = 1000 # le nombre d'essai
interest_rate = 0.01 # les tx d'intérêts 
dt = 1/12 # pas mensuel

In [9]:
class Option_pricer:
    
    def volatility(self,maturity):
        if maturity ==1:
            volatility = 0.15
        elif maturity == 2:
            volatility = 0.2
        return volatility
  
    def lookback_call_bs(self,price,strike,maturity,volatility,interest_rate):
        x, var = 0, 0
        min_st = 100
        IC = [0,0]
        for i in range(N):
            Ft = price
            for i in range(int(maturity/dt)):
                dW = gauss(0.0,sqrt(dt))
                Ft = Ft * exp((interest_rate-0.5*volatility**2)*dt + volatility*dW)
            if Ft < min_st :
                min_st = Ft
            Prix = exp(-interest_rate*maturity)*max(Ft-min_st,0)  
            x += Prix
            var += Prix**2
        var /= N
        x /= N
        var -= x**2
        IC[1] = x + 2*sqrt(var/N)
        IC[0] = x - 2*sqrt(var/N)
        return x, IC, var
  
    def __init__(
    self, price, strike, volatility, maturity,
    interest_rate, dt):
        self.price = price
        self.strike = strike
        self.volatility = self.volatility(maturity)
        self.maturity = maturity
        self.interest_rate = interest_rate
        self.dt = dt
        self.price_lookback_call,self.IClb,self.sdlb = self.lookback_call_bs(price,strike,maturity,volatility,interest_rate)

In [10]:
ec = Option_pricer(price,strike,volatility,maturity,interest_rate,dt)
print("prix :" ,ec.price_lookback_call,ec.IClb,ec.sdlb)

prix : 55.79976830344001 [53.90109949788715, 57.69843710899287] 901.2358082948808


#  BINOMIAL

In [11]:
#CONTEXTE
dt = 1/12
price  = 30
maturity = 2
volatility = 0.001
strike = 30
interest_rate = 0.005
N =  int(maturity/dt)

In [12]:
def combos(n, i):
    return math.factorial(n) / (math.factorial(n-i)*math.factorial(i))
    
def binom_EU1(price, strike , maturity, interest_rate, volatility, N):
    dt = maturity/N
    u = np.exp(volatility * np.sqrt(dt))
    d = np.exp(-volatility * np.sqrt(dt))
    p = (  np.exp(interest_rate*dt) - d )  /  (  u - d )
    S0 = price
    value = 0
    for i in range(N+1):
        node_prob = combos(N, i)*p**i*(1-p)**(N-i)
        ST = S0*(u)**i*(d)**(N-i)
        value += max(ST-strike,0) * node_prob
            
    return value*np.exp(-interest_rate*maturity)


binom_EU1(price,strike,maturity,interest_rate, volatility,N)

0.29758067780838965

# OPTION BERMUDIENNE